Here is the python notebook to create isolated indices. Just make sure you put the newest enddate in the cell below before running this notebook. 

In [110]:
from __future__ import division
import os
import os.path
import psycopg2
import smtplib
import datetime
import numpy as np
import pandas as pd
import re
import json
import calendar
import math
import sys
import collections
import matplotlib as mt
%matplotlib inline

from xlsxwriter.utility import xl_rowcol_to_cell
from xlsxwriter.utility import xl_col_to_name
from xlsxwriter.workbook import Workbook

sys.path.append("../khan")
sys.path.append("../modules")

In [111]:
# Establish Connection
cnxn = psycopg2.connect(host='sevenpdredshift.cgxitduwhaqr.us-east-1.redshift.amazonaws.com', database='sevenpark', port='5439', user='sevenpuser', password='7Parkdata', sslmode='require')
cur = cnxn.cursor()

# Set enddate each time you run these reports


In [112]:




enddate = "2019-01-31"






In [113]:
Alist_dict = {'merchant': 'Vail Resorts',
                 'custom_filter': " ",
                 'price_filter': " ",
                 'ticker': 'MTN',
                 'metric':'Sales',
                 'start_date':"2013-11-01",
                 'end_date': enddate,
                 'tablename': 'MTN_Alist'}
Epic_dict = {'merchant': 'Vail Resorts',
                 'custom_filter': " AND itemdescription ILIKE '%epic%' AND itemdescription NOT ILIKE '%epic%day%' ",
                 'price_filter': "  ",
                 'ticker': 'MTN',
                 'metric':'Sales',
                 'start_date':"2013-11-01",
                 'end_date': enddate,
                 'tablename': 'Epic_Pass'}

In [114]:
merchantdaily_basequery = '''DROP TABLE IF EXISTS khan.kevintemp_{tablename};

CREATE TABLE khan.kevintemp_{tablename} (
  "udate" date,
  "cohort" varchar(255),
  "merchant" varchar(100),
  "ticker" varchar(25),
  "quality" varchar(3),
  "vol" int4,
  "rev" numeric(11,2),
  "panel" int4,
  "shiptozip" varchar(10),
  "msa" varchar(255),
  "uname" varchar(255),
  "region" varchar(20)
);

INSERT INTO khan.kevintemp_{tablename} 



SELECT
    orderdate AS udate,
    t0.cohort,
    merchant,
    ticker,
    NULL,
    COUNT(*) AS vol,
    SUM(sales) AS rev,
    panel,
    shiptozip,
    msa,
    uname,
    region
FROM
(
    SELECT
        cohort,
        rawdata.merchant,
        ticker,
        orderdate,
        ordertotal AS sales,
        orderid,
        userzips.shiptozip,
        msa,
        uname,
        region
    FROM
    (
        SELECT
            merchant,
            mailboxid,
            orderdate,
            emailsent,
            ordertotal,
            orderid
        FROM 
            khan.rawdata
        WHERE merchant IN ('{merchant}') {custom_filter}
        AND mailboxid NOT IN (SELECT DISTINCT mboxid FROM khan.mailbox_status WHERE panelexitdate = '2014-7-2' or panelentrydate = '2011-6-1')
        GROUP BY 
            merchant,
            mailboxid,
            orderdate,
            emailsent,
            ordertotal,
            orderid
    ) AS rawdata
    INNER JOIN (SELECT * FROM khan.merchantinfo WHERE merchant = '{merchant}') AS merchantinfo
         ON rawdata.merchant=merchantinfo.merchant and rawdata.orderdate >= merchantinfo.effective_start_date and rawdata.orderdate <= merchantinfo.effective_end_date
    INNER JOIN (SELECT * FROM khan.cohort_list) AS cohort_list
        ON rawdata.mailboxid = cohort_list.mailboxid
            AND rawdata.orderdate = cohort_list.udate
    INNER JOIN (SELECT mailboxid, SUBSTRING(shiptozip FROM 0 FOR 6) AS shiptozip FROM khan.userzips WHERE mailboxid IN (SELECT mailboxid FROM khan.userzips GROUP BY mailboxid HAVING COUNT(*) = 1)) AS userzips
        ON rawdata.mailboxid=userzips.mailboxid
    LEFT JOIN sundance.ziptomsa
        ON userzips.shiptozip=ziptomsa.zipcode
    LEFT JOIN sundance.statetoregion
        ON ziptomsa.state=statetoregion.stateabbr
    {price_filter}
    GROUP BY
        cohort,
        ticker,
        rawdata.merchant,
        orderdate,
        emailsent,
        ordertotal,
        orderid,
        userzips.shiptozip,
        msa,
        uname,
        region
) AS t0
INNER JOIN khan.cohort_panels
    ON t0.orderdate=cohort_panels.udate
        AND t0.cohort = cohort_panels.cohort
GROUP BY
    orderdate,
    t0.cohort,
    merchant,
    ticker,
    panel,
    shiptozip,
    msa,
    uname,
    region; 
'''



In [115]:
## Quarterly Query

quarterlyquery = """
    SELECT
        fiscalyear,
        fiscalquarter,
        'Q'||fiscalquarter||' '||fiscalyear AS quarter,
        CASE
            WHEN quarterend > '{end_date}' THEN '{end_date}' - quarterstart + 1
            ELSE quarterend - quarterstart + 1
        END AS numDays,
        ISNULL(SUM (rev_idx),0) AS rev,
        ISNULL(SUM (vol_idx),0) AS vol,
        quarterstart,
        quarterend,
        ticker,
        ISNULL(metricvalue, -1) AS metricvalue,
        metricname
    FROM
    (
        SELECT
            udate,
            sum(vol_idx) AS vol_idx,
            sum(rev_idx) AS rev_idx
        FROM
        (
            SELECT
                udate AS udate,
                vol,
                rev,
                panel,
                vol * 1000.0/panel as vol_idx,
                rev * 1000.0/panel as rev_idx
            FROM
                khan.kevintemp_{tablename}
            WHERE cohort = 'all'
            AND merchant IN ('{merchant}')
        )
        WHERE udate BETWEEN '{start_date}' AND '{end_date}'
        GROUP BY udate
    ) AS main
    INNER JOIN (SELECT * FROM sundance.reportedmetrics WHERE metricname = '{metric}' AND ticker = '{ticker}') AS reportedmetrics
        ON main.udate >= reportedmetrics.quarterstart AND main.udate <= reportedmetrics.quarterend
    GROUP BY
        fiscalyear,
        fiscalquarter,
        quarterstart,
        quarterend,
        ticker,
        metricvalue,
        metricname
    ORDER BY
        quarterstart;
"""


In [116]:
merchantweekly_base_query = """
SELECT
      t1.week_start,
      t1.week_end,
      ISNULL(rev_idx,0) as rev,
      ISNULL(vol_idx,0) as vol
  FROM
  (SELECT
      days - 6 AS week_start,
      days AS week_end
  FROM
      sundance.alldays
  WHERE
      days >= '{start_date}'
  AND days <= '{end_date}'
  AND dow = 6
  ORDER BY week_start) AS t1
  LEFT JOIN
  (
      SELECT
          week_end,
          sum(vol_idx) AS vol_idx,
          sum(rev_idx) AS rev_idx
      FROM
      (
          SELECT
          --  udate,
              CASE
                  WHEN EXTRACT (DOW FROM udate :: TIMESTAMP) = 0 THEN date(udate + INTERVAL '6 days')
                  WHEN EXTRACT (DOW FROM udate :: TIMESTAMP) = 1 THEN date(udate + INTERVAL '5 days')
                  WHEN EXTRACT (DOW FROM udate :: TIMESTAMP) = 2 THEN date(udate + INTERVAL '4 days')
                  WHEN EXTRACT (DOW FROM udate :: TIMESTAMP) = 3 THEN date(udate + INTERVAL '3 days')
                  WHEN EXTRACT (DOW FROM udate :: TIMESTAMP) = 4 THEN date(udate + INTERVAL '2 days')
                  WHEN EXTRACT (DOW FROM udate :: TIMESTAMP) = 5 THEN date(udate + INTERVAL '1 days')
                  ELSE udate
              END AS week_end,
              vol,
              rev,
              panel,
              vol * 1000.0/panel as vol_idx,
              rev * 1000.0/panel as rev_idx
          FROM
              khan.kevintemp_{tablename}
          WHERE merchant = '{merchant}'
          AND cohort = 'all'
      )
      WHERE week_end BETWEEN '{start_date}' AND '{end_date}'
      GROUP BY week_end
  ) AS main
  ON t1.week_end=main.week_end
    ORDER BY week_end;
"""

In [117]:
dict_list = [Alist_dict, Epic_dict]

In [118]:
output_queries = []
for dictt in dict_list:
    final = merchantdaily_basequery.format(**dictt)
    cur.execute(final)
    cnxn.commit()

## Pulling Weekly Index Values

In [119]:
weeklydata = {}
for dictt in dict_list:
    query = merchantweekly_base_query.format(**dictt)
    weeklydata[dictt["tablename"]] = pd.read_sql(query, cnxn)

## Pulling Quarterly Index Values

In [120]:
quarterlydata = {}
for dictt in dict_list:
    query = quarterlyquery.format(**dictt)
    quarterlydata[dictt["tablename"]] = pd.read_sql(query, cnxn)

## Quarterly Data Output file
#### This is saved locally in the same file as this notebook 

In [121]:
quarterlyworkbook = Workbook("MTN_quarterly_data.xlsx") #Create Workbook

for merchant in quarterlydata:
    df = quarterlydata[merchant]
    worksheet = quarterlyworkbook.add_worksheet(merchant) #Create Worksheet
    for j in range(len(df.columns.values)):
        worksheet.write(0, j, df.columns.values[j])
        for i in range(len(df)):
            worksheet.write(1 + i, j, df.iloc[i][j]) #Loop through rows and columns to enter into cells


quarterlyworkbook.close() #Close Workbook

## Weekly Data Output file
#### This is saved locally in the same file as this notebook 

In [122]:
weeklyworkbook = Workbook("MTN_weekly_data.xlsx") #Create Workbook

for merchant in weeklydata:
    df = weeklydata[merchant]
    worksheet = weeklyworkbook.add_worksheet(merchant) #Create Worksheet
    for j in range(len(df.columns.values)):
        worksheet.write(0, j, df.columns.values[j])
        for i in range(len(df)):
            worksheet.write(1 + i, j, df.iloc[i][j]) #Loop through rows and columns to enter into cells


weeklyworkbook.close() #Close Workbook

Make sure you download both of these files and email them ALONG with their corresponding reports that they send 